In [1]:
import pytorch_tabular

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("newest_data.csv")

In [4]:
df

,cc_num,merchant,category,amt,first,last,gender,street,city,state,...,total_transaction_amt,trans_year,trans_month,trans_day,trans_hour,trans_minute,trans_second,dob_year,dob_month,dob_day
0,4.658610e+12,fraud_Subramaniam-Goyal Pvt Ltd,online_shopping,8548.59,Divit,Tiwari,M,"07/547, Baral Chowk, Solapur-734676",Berhampur,Uttarakhand,...,51291.540000,2023,3,21,11,41,0,1986,7,23
1,4.041460e+12,fraud_Agrawal Group Pvt Ltd,online_shopping,7012.15,Himmat,Basak,F,"H.No. 721, Johal Street, Gaya 138332",Shimla,Jharkhand,...,63109.350000,2022,10,20,2,16,0,1954,7,14
2,4.273900e+15,"fraud_Hora, Tella and Bawa Pvt Ltd",online_shopping,3294.28,Tejas,Soni,F,"728, Anne Road, Madhyamgram-169150",Shahjahanpur,Punjab,...,113009.360000,2023,3,21,11,41,0,1996,7,27
3,3.530110e+15,"fraud_Devan, Iyengar and Badami Pvt Ltd",online_shopping,3143.95,Shalv,Kata,F,"30/10, Mane Road, Faridabad 973707",Satna,Sikkim,...,82285.272863,2022,6,12,3,47,0,1996,2,6
4,4.791230e+15,"fraud_Date, Kurian and Sule Pvt Ltd",online_shopping,738.24,Miraan,Srivastava,F,"84/474, Baria Road, Adoni-525538",Davanagere,Tripura,...,4429.440000,2024,2,11,15,15,0,1965,10,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,3.573230e+15,"fraud_Chanda, Gade and Vaidya Pvt Ltd",travel,1054.76,Tejas,Srinivasan,M,"43/89, Bumb Road, Jodhpur-140664",Aurangabad,Uttarakhand,...,11743.720000,2023,3,21,11,41,0,1990,10,9
6566,3.481050e+14,fraud_Tripathi-Karpe Pvt Ltd,travel,7629.93,Ranbir,Bora,F,"85/219, Ramesh Nagar, Pondicherry 277735",Giridih,Tamil Nadu,...,38149.650000,2023,12,28,0,19,0,1954,2,16
6567,4.117590e+12,fraud_Lalla PLC Pvt Ltd,travel,8697.49,Piya,Dave,F,"H.No. 472, Din Circle, Miryalaguda 438331",Aurangabad,Sikkim,...,99376.400000,2022,6,21,4,54,0,1952,6,28
6568,4.572190e+18,fraud_Lalla PLC Pvt Ltd,travel,7771.61,Zeeshan,Zacharia,F,"07/16, Chakraborty Circle, Medininagar 534484",Faridabad,Tamil Nadu,...,157003.770000,2022,11,22,5,8,0,1987,1,6


In [5]:
cat_col_names=df.select_dtypes(include=["object"]).columns.tolist()
num_col_names=df.select_dtypes(include=["int64","float64"]).columns.tolist()

In [6]:
num_col_names.remove("is_fraud")

In [7]:
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from pytorch_tabular import TabularModel

In [8]:
data_config = DataConfig(
    target=["is_fraud"],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names
)
optimiser_config=OptimizerConfig()

trainer_config = TrainerConfig(
    batch_size=1024,  # Batch size for training
    max_epochs=100,  # Maximum number of epochs
)

In [9]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,random_state=42,test_size=0.2)

In [10]:
%%time
from pytorch_tabular import model_sweep
import warnings

# Filtering out the warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task="classification",  # One of "classification", "regression"
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimiser_config,
        trainer_config=trainer_config,
        model_list="standard",
        metrics=["accuracy", "recall"],
        metrics_params=[{}, {"average": "macro"}],
        metrics_prob_input=[False, True],
        rank_metric=("accuracy", "higher_is_better"),
        progress_bar=True,
        verbose=True,
        suppress_lightning_logger=True,
    )

Output()

2024-07-18 10:43:14,480 - {pytorch_tabular:329} - INFO - Training CategoryEmbeddingModel

2024-07-18 10:43:44,328 - {pytorch_tabular:368} - INFO - Finished Training CategoryEmbeddingModel

2024-07-18 10:43:44,329 - {pytorch_tabular:369} - INFO - Results: model: CategoryEmbeddingModel, # Params: 438 T,  
epochs: 14, test_loss: 0.11002220958471298, test_accuracy: 0.9611872434616089, test_recall: 0.9608750939369202,    
time_taken: 29.869801998138428, time_taken_per_epoch: 2.133557285581316

2024-07-18 10:43:44,375 - {pytorch_tabular:329} - INFO - Training CategoryEmbeddingModel

...

2024-07-18 10:47:12,746 - {pytorch_tabular:368} - INFO - Finished Training FTTransformerModel

2024-07-18 10:47:12,749 - {pytorch_tabular:369} - INFO - Results: model: FTTransformerModel, # Params: 297 T,      
epochs: 14, test_loss: 0.16167977452278137, test_accuracy: 0.9474886059761047, test_recall: 0.9478395581245422,    
time_taken: 107.1849479675293, time_taken_per_epoch: 7.6560677119663785

2024-07-18 10:47:12,752 - {pytorch_tabular:387} - INFO - Model Sweep Finished

2024-07-18 10:47:12,754 - {pytorch_tabular:388} - INFO - Best Model: CategoryEmbeddingModel

CPU times: user 2min 46s, sys: 1min 19s, total: 4min 5s
Wall time: 3min 58s


In [11]:
sweep_df

,model,# Params,epochs,test_loss,test_accuracy,test_recall,time_taken,time_taken_per_epoch,params
1,CategoryEmbeddingModel,590 T,12,0.086244,0.967276,0.967501,6.634397,0.552866,"{'task': 'classification', 'head': 'LinearHead..."
0,CategoryEmbeddingModel,438 T,14,0.110022,0.961187,0.960875,29.869802,2.133557,"{'task': 'classification', 'head': 'LinearHead..."
3,GANDALFModel,19 M,10,0.231950,0.958904,0.958780,30.456507,3.045651,"{'task': 'classification', 'head': 'LinearHead..."
2,GANDALFModel,8 M,7,0.199940,0.950533,0.950133,14.033805,2.004829,"{'task': 'classification', 'head': 'LinearHead..."
6,FTTransformerModel,297 T,14,0.161680,0.947489,0.947840,107.184948,7.656068,"{'task': 'classification', 'head': 'LinearHead..."
4,TabNetModel,388 T,5,0.716230,0.538813,0.539463,28.672799,5.734560,"{'task': 'classification', 'head': 'LinearHead..."
5,TabNetModel,470 T,4,0.762224,0.518265,0.515326,21.375667,5.343917,"{'task': 'classification', 'head': 'LinearHead..."


In [12]:
sweep_df.drop(columns=["params", "time_taken", "epochs"]).style.background_gradient(
    subset=["test_accuracy", "test_recall"], cmap="RdYlGn"
).background_gradient(subset=["time_taken_per_epoch", "test_loss"], cmap="RdYlGn_r")

,model,# Params,test_loss,test_accuracy,test_recall,time_taken_per_epoch
1,CategoryEmbeddingModel,590 T,0.086244,0.967276,0.967501,0.552866
0,CategoryEmbeddingModel,438 T,0.110022,0.961187,0.960875,2.133557
3,GANDALFModel,19 M,0.231950,0.958904,0.958780,3.045651
2,GANDALFModel,8 M,0.199940,0.950533,0.950133,2.004829
6,FTTransformerModel,297 T,0.161680,0.947489,0.947840,7.656068
4,TabNetModel,388 T,0.716230,0.538813,0.539463,5.734560
5,TabNetModel,470 T,0.762224,0.518265,0.515326,5.343917


In [10]:
from rich import print
from rich.pretty import pprint

In [11]:
from pytorch_tabular import MODEL_SWEEP_PRESETS

In [15]:
pprint(list(MODEL_SWEEP_PRESETS["high_memory"]))

[
│   'AutoIntConfig',
│   'CategoryEmbeddingModelConfig',
│   'DANetConfig',
│   'FTTransformerConfig',
│   'GANDALFConfig',
│   'GatedAdditiveTreeEnsembleConfig',
│   'NodeConfig',
│   'TabNetModelConfig',
│   'TabTransformerConfig'
]

In [12]:
model_list=[
   'AutoIntConfig',
   'CategoryEmbeddingModelConfig',
   'DANetConfig',
   'GANDALFConfig',
   'GatedAdditiveTreeEnsembleConfig',
   'TabNetModelConfig',
   'TabTransformerConfig'
]

In [13]:
from pytorch_tabular.models import (
    AutoIntConfig,
    CategoryEmbeddingModelConfig,
    DANetConfig,
    
    GANDALFConfig,
    GatedAdditiveTreeEnsembleConfig,
    
    TabNetModelConfig,
    TabTransformerConfig,
)

In [14]:
model_list

['AutoIntConfig',
 'CategoryEmbeddingModelConfig',
 'DANetConfig',
 'GANDALFConfig',
 'GatedAdditiveTreeEnsembleConfig',
 'TabNetModelConfig',
 'TabTransformerConfig']

In [18]:
%%time
from pytorch_tabular import model_sweep
from pytorch_tabular.models import (
    AutoIntConfig,
    CategoryEmbeddingModelConfig,
    DANetConfig,
    FTTransformerConfig,
    GANDALFConfig,
    GatedAdditiveTreeEnsembleConfig,
    NodeConfig,
    TabNetModelConfig,
    TabTransformerConfig,
)
import warnings

# Define the task
task = "classification"

# Construct the model list manually with the task argument
model_list = [
    AutoIntConfig(task=task),
    CategoryEmbeddingModelConfig(task=task),
    DANetConfig(task=task),
    GANDALFConfig(task=task),
    TabNetModelConfig(task=task),
    TabTransformerConfig(task=task),
]

# Filtering out the warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task=task,  # One of "classification", "regression"
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimiser_config,  # Ensure correct spelling of optimizer_config
        trainer_config=trainer_config,
        model_list=model_list,  # Pass the manually constructed model list
        metrics=["accuracy", "f1_score"],  # Metrics to evaluate
        metrics_params=[{}, {"average": "macro"}],  # Parameters for the metrics
        metrics_prob_input=[False, True],  # Whether the metrics require probability inputs
        rank_metric=("accuracy", "higher_is_better"),  # Metric to rank the models by
        progress_bar=True,  # Show a progress bar
        verbose=True,  # Output verbose logs
        suppress_lightning_logger=True,  # Suppress the lightning logger
    )


Output()

2024-07-18 12:58:07,725 - {pytorch_tabular:329} - INFO - Training AutoIntModel

2024-07-18 12:58:38,389 - {pytorch_tabular:368} - INFO - Finished Training AutoIntModel

2024-07-18 12:58:38,391 - {pytorch_tabular:369} - INFO - Results: model: AutoIntModel, # Params: 108 T, epochs: 27,
test_loss: 0.14365410804748535, test_accuracy: 0.965753436088562, test_f1_score: 0.9657127261161804, time_taken:   
30.688654899597168, time_taken_per_epoch: 1.1366168481332284

2024-07-18 12:58:38,486 - {pytorch_tabular:329} - INFO - Training CategoryEmbeddingModel

...

2024-07-18 13:00:28,748 - {pytorch_tabular:368} - INFO - Finished Training TabTransformerModel

2024-07-18 13:00:28,750 - {pytorch_tabular:369} - INFO - Results: model: TabTransformerModel, # Params: 451 T,     
epochs: 16, test_loss: 0.2215551733970642, test_accuracy: 0.9512937664985657, test_f1_score: 0.9511160254478455,   
time_taken: 59.09898519515991, time_taken_per_epoch: 3.6936865746974945

2024-07-18 13:00:28,757 - {pytorch_tabular:387} - INFO - Model Sweep Finished

2024-07-18 13:00:28,759 - {pytorch_tabular:388} - INFO - Best Model: AutoIntModel

CPU times: user 1min 12s, sys: 41.4 s, total: 1min 53s
Wall time: 2min 21s


In [19]:
sweep_df

,model,# Params,epochs,test_loss,test_accuracy,test_f1_score,time_taken,time_taken_per_epoch,params
0,AutoIntModel,108 T,27,0.143654,0.965753,0.965713,30.688655,1.136617,"{'task': 'classification', 'head': 'LinearHead..."
1,CategoryEmbeddingModel,347 T,21,0.128371,0.961187,0.961133,6.860123,0.326673,"{'task': 'classification', 'head': 'LinearHead..."
5,TabTransformerModel,451 T,16,0.221555,0.951294,0.951116,59.098985,3.693687,"{'task': 'classification', 'head': 'LinearHead..."
3,GANDALFModel,8 M,7,0.202682,0.947489,0.947416,7.704914,1.100702,"{'task': 'classification', 'head': 'LinearHead..."
4,TabNetModel,301 T,5,0.694355,0.520548,0.506689,15.493516,3.098703,"{'task': 'classification', 'head': 'LinearHead..."
2,DANetModel,3 M,3,0.695688,0.503044,0.499561,21.130135,7.043378,"{'task': 'classification', 'head': 'LinearHead..."


In [21]:
sweep_df.drop(columns=["params", "time_taken", "epochs"]).style.background_gradient(
    subset=["test_accuracy", "test_f1_score"], cmap="RdYlGn"
).background_gradient(subset=["time_taken_per_epoch", "test_loss"], cmap="RdYlGn_r")

,model,# Params,test_loss,test_accuracy,test_f1_score,time_taken_per_epoch
0,AutoIntModel,108 T,0.143654,0.965753,0.965713,1.136617
1,CategoryEmbeddingModel,347 T,0.128371,0.961187,0.961133,0.326673
5,TabTransformerModel,451 T,0.221555,0.951294,0.951116,3.693687
3,GANDALFModel,8 M,0.202682,0.947489,0.947416,1.100702
4,TabNetModel,301 T,0.694355,0.520548,0.506689,3.098703
2,DANetModel,3 M,0.695688,0.503044,0.499561,7.043378


In [26]:
type(sweep_df)

pandas.core.frame.DataFrame

In [27]:
daf=sweep_df[["model","test_accuracy","test_f1_score"]]

In [28]:
daf

,model,test_accuracy,test_f1_score
0,AutoIntModel,0.965753,0.965713
1,CategoryEmbeddingModel,0.961187,0.961133
5,TabTransformerModel,0.951294,0.951116
3,GANDALFModel,0.947489,0.947416
4,TabNetModel,0.520548,0.506689
2,DANetModel,0.503044,0.499561
